# Course counts

Drop this in a directory that contains a bunch of UNCA class schedule CSVs downloaded from https://www.unca.edu/schedules/; this notebook reads through those and builds a file with some information on my courses.

In [1]:
import pandas as pd
import os
import re

In [15]:
pattern = re.compile(r"^Courses (FALL|SPRING|SUMMER) (\d{4})\.csv$")
data = pd.DataFrame()
for course in os.listdir():
    match = pattern.match(course)
    if match:
        sem, year = match.groups()
        result = {'SEM': sem, 'YEAR': year}
        df = pd.read_csv(course)
        df = df[['Code', 'Department', 'Title', 'Instructors', 'Hours', 'EnrollmentCurrent']]
        df = df[df['EnrollmentCurrent'] > 0]
        df = df[df['Hours'] > 0]
        df['Semester'] = sem
        df['Year'] = year
        data = pd.concat([data, df], ignore_index=True)
data.sort_values(by=['Year', 'Semester'], inplace=True)
data = data[data.apply(lambda r: r.Instructors == "McClure, Mark", axis=1)]
data['Hours'] = data['Hours'].astype(int)
data['Enrollment'] = data['EnrollmentCurrent'].astype(int)
data = data[['Code', 'Department', 'Title', 'Hours', 'Enrollment', 'Semester', 'Year']]
data['Title'] = data.Title.apply(lambda s: s.split(':')[-1].strip())

data.to_csv('courses.csv', index=False, encoding='utf-8-sig')
